In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
import tsplib95

In [2]:
problem = tsplib95.load('../data/bays29.tsp')

In [5]:
peso = np.copy(problem.edge_weights)
masked = np.ma.masked_equal(peso, 0, copy=False)
masked = np.ma.masked_equal(peso, 0, copy=False)
peso[16][21]

77

In [8]:
#calculate distance
list(problem.get_nodes())
edge1 = 6
edge2 = 7
dist = np.min(problem.get_weight(edge1,edge2))
dist

386

### I discover that I can make a deepcopy of the array, but just to remember I'll let this file here.

In [7]:
weights = np.copy(problem.edge_weights)
weight_copy = copy.deepcopy(weights)